In [3]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from shapley import ShapleyIndices, ShapleyKrigingIndices
from shapley.tests import Ishigami, AdditiveGaussian
from shapley.plots import set_style_paper, plot_sensitivity_results
from shapley.base import ProbabilisticModel 
from shapley.kriging import KrigingModel

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

savefigs = True
ylim = [0., 1.]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
mu_betas = [247, 218, 148, 105]
sigma_betas = [6, 5, 4, 3]
mu_normal = [35, 18, 14, 11, 8, 1, -31]
sigma_normal = [5, 5, 5, 5, 5, 5, 5]
bounds_low = [235, 207, 141, 100]
bounds_up = [259, 229, 155, 110]

margins = []
for mu, sigma, a, b in zip(mu_betas, sigma_betas, bounds_low, bounds_up):
    dist = ot.Beta()
    dist.setParameter(ot.BetaMuSigma()([mu, sigma, a, b]))
    margins.append(dist)
    
for mu, sigma in zip(mu_normal, sigma_normal):
    margins.append(ot.Normal(mu, sigma))
    
mu = np.asarray(mu_betas + mu_normal)
sigma = np.asarray(sigma_betas + sigma_normal)

In [53]:
correlation = ot.CorrelationMatrix(pd.read_csv('./data/correlation.csv', sep=',', header=None).values)
copula = ot.NormalCopula(correlation)
input_distribution = ot.ComposedDistribution(margins, copula)

In [90]:
data = pd.read_csv('./data/results_unif.txt', sep=' ', header=None, usecols=range(12))
X = data.values[:, :11]
y = data.values[:, 11]
#X = (X - mu) / sigma
#y = (y - y.mean()) / y.std()

In [109]:
library = 'sklearn'

meta_model = KrigingModel(model=lambda x: None, input_distribution=input_distribution)
meta_model.input_sample = X
meta_model.output_sample = y
meta_model.build(library=library)

In [ ]:
score_q2_loo = meta_model.compute_score_q2_loo()
print('Leave One Out Q2 score:', score_q2_loo)

In [ ]:
Krigi